# Dynamic Pricing for Urban Parking Lots
## Introduction


**This project builds a dynamic pricing of urban parking lots using real-time data and economic principles.** Given 14 parking locations and multiple time-varying inputs (like occupancy, queue length, vehicle type, traffic, and special events), the objective is to create a pricing model that adjusts the parking price intelligently over time.

The model ingests streaming data(obtained from csv file), evaluate demand and competition, and update prices accordingly using custom logic built only with **NumPy, Pandas, and Pathway**.


In [1]:
# Install packages pathway and bokeh
!pip install pathway bokeh --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.1/319.1 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [15]:
#Import libraries
import numpy as np
import pandas as pd
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

# Step 1: Importing and Preprocessing the Data

In [16]:
# read dataset as Pandas DataFrame
df = pd.read_csv('/content/dataset.csv')

In [17]:
# Combine the 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

# Encoding categorical data
df['VehicleTypeWeight'] = df['VehicleType'].map({'cycle':0.1, 'bike': 0.5, 'car': 1.0, 'truck': 1.5})
df['TrafficLevel'] = df['TrafficConditionNearby'].map({'low': 0, 'average': 1, 'high': 2})


# Compute occupancy ratio
df["OccRatio"] = df["Occupancy"] / df["Capacity"]



# Initialize Model 1 price list
model where the next price is a function of the previous price and current occupancy

In [18]:
# Initialize Model 1 price list
prices = []


α = 0.6         #tunable parameter
base_price = 10

# First price: apply model to first row directly
first_price = base_price + α * df.loc[0, "OccRatio"]
prices.append(round(first_price, 4))

# Recursive pricing for remaining rows
for i in range(1, len(df)):
    prev_price = prices[-1]
    occ_ratio = df.loc[i, "OccRatio"]
    price = prev_price + α * occ_ratio
    prices.append(round(price, 4))

df["Price_Model1"] = prices


# Sort the DataFrame by the new 'Timestamp' and 'SystemCodeNumber' column and reset the index
df = df.sort_values(by=['SystemCodeNumber', 'Timestamp']).reset_index(drop=True)

In [19]:
# Save the selected columns to a CSV file for streaming or downstream processing
df[["SystemCodeNumber", "Occupancy", "Capacity", "OccRatio","QueueLength", "TrafficLevel", "IsSpecialDay", "VehicleTypeWeight", "Timestamp","Price_Model1"]].to_csv("parking_stream.csv", index=False)



In [21]:
# Define the schema for the streaming data using Pathway
# This schema specifies the expected structure of each data row in the stream

class ParkingSchema(pw.Schema):
    Timestamp: str   # Timestamp of the observation (should ideally be in ISO format)
    Occupancy: int   # Number of occupied parking spots
    Capacity: int    # Total parking capacity at the location
    SystemCodeNumber: str  #lot unique number
    QueueLength: int   #queue length
    TrafficLevel: int  #Traffic level
    IsSpecialDay: int   #usual working day(0) and holiday(1)
    VehicleTypeWeight: float  #type of vehicle
    OccRatio: float            #occupancy/capacity
    Price_Model1: float        #recursive prices of Model 1



In [22]:
# Load the data as a simulated stream using Pathway's replay_csv function
# This replays the CSV data at a controlled input rate to mimic real-time streaming
# input_rate=1000 means approximately 1000 rows per second will be ingested into the stream.

data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)


In [23]:
# Define the datetime format to parse the 'Timestamp' column
fmt = "%Y-%m-%d %H:%M:%S"

# Add new columns to the data stream:
# - 't' contains the parsed full datetime
# - 'day' extracts the date part and resets the time to midnight (useful for day-level aggregations)
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)


# Step 2: Pricing Models 1 and 2

In [24]:
α = 0.6 #tunable parameter: 0.6 as used in model 1


# ---------- Model 1 ----------
model1 = data_with_time.with_columns(
    Price_Model1 = pw.this.Price_Model1
)


# ---------- Model 2 ----------
# tunable parameters(can be modified as per our intuition)

λ = 0.7
β = 0.3
γ = 0.2
δ = 0.5
ε = 0.4

# Step 1: Calculate RawDemand
model2_step1 = data_with_time.with_columns(
    RawDemand = α * (pw.this.Occupancy / pw.this.Capacity) +
                β * pw.this.QueueLength -
                γ * pw.this.TrafficLevel +
                δ * pw.this.IsSpecialDay +
                ε * pw.this.VehicleTypeWeight

)

# Step 2: Get min and max from RawDemand using reduce
raw_demand_stats = model2_step1.reduce(
    MaxDemand = pw.reducers.max(pw.this.RawDemand),
    MinDemand = pw.reducers.min(pw.this.RawDemand),
)

# Step 3: Join  min/max back to all rows
model2_step2 = model2_step1.join(raw_demand_stats).select(
    **pw.left,  # keep original columns
    MaxDemand = pw.right.MaxDemand,
    MinDemand = pw.right.MinDemand
)

# Step 4: Normalize RawDemand (Min-Max Normalization)
model2_step3 = model2_step2.with_columns(
    NormDemand = pw.apply(
        lambda r, rmin, rmax: 0.0 if rmax == rmin else max(0.0, min(1.0, (r - rmin) / (rmax - rmin))),
        pw.this.RawDemand,
        pw.this.MinDemand,
        pw.this.MaxDemand
    )
)

# Step 5: Compute bounded price using NormDemand
model2 = model2_step3.with_columns(
    Price_Model2 = pw.apply(
        lambda d: max(5.0, min(20.0, 10 * (1 + λ * d))),
        pw.this.NormDemand
    )
)
# uncomment below to see model 1 and model 2 prices
# pw.debug.compute_and_print(model2)

# Step 3: Visualizing Daily Price Fluctuations with a Bokeh Plot
### Model 1 vs Model 2 for each lot

In [25]:
pn.extension()

# ---------- Visualization ----------
def make_plot_m1(source, price_col, lot_id, model_name):
    fig = bokeh.plotting.figure(
        height=250,
        width=600,
        title=f"{model_name} - {lot_id}",
        x_axis_type="datetime"
    )
    fig.line("t", price_col, source=source, line_width=2, color="navy")
    fig.scatter("t", price_col, source=source, size=4, color="red")
    return fig
def make_plot_m2(source, price_col, lot_id, model_name):
    fig = bokeh.plotting.figure(
        height=250,
        width=600,
        title=f"{model_name} - {lot_id}",
        x_axis_type="datetime"
    )
    fig.line("t", price_col, source=source, line_width=2, color="green")
    fig.scatter("t", price_col, source=source, size=4, color="orange")
    return fig

# Get all unique parking lot IDs
lots = df["SystemCodeNumber"].unique().tolist()

# Generate plots
plots = []
for lot in lots:
    m1_plot = model1.filter(pw.this.SystemCodeNumber == lot).plot(
        lambda src: make_plot_m1(src, "Price_Model1", lot, "Model 1"), sorting_col="t")
    m2_plot = model2.filter(pw.this.SystemCodeNumber == lot).plot(
        lambda src: make_plot_m2(src, "Price_Model2", lot, "Model 2"), sorting_col="t")
    plots.append(pn.Row(m1_plot, m2_plot))

#Visualize dashboard
dashboard = pn.Column("# Real-Time Pricing for All Parking Lots: Model 1 vs Model 2", *plots)
dashboard

Column
    [0] Markdown(str)
    [1] Row
        [0] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [2] Row
        [0] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [3] Row
        [0] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [4] Row
        [0] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [5] Row
        [0] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [6] Row
        [0] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [7] Row
        [0] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [8] Row
        [0] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [9] Row
        [0] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [10] Row
        [0] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [11] Row
        [0] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [12] Row
        [0] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
        [1] Column
            [0] Row
                [0] Markdown(str)
                [1] Toolt

# STEP 5: MODEL 3 - Competitive Pricing Model
Here we took lot "**BHMBCCTHL01**" as our lot and "**BHMBCCMKT01**" as competitor lot because of their proximity calculated using their respective longitude and latitude and Haversine formula. **"BHMBCCTHL01"** was a preffered choice because of having many instances of lot being full (occupancy > capacity).
#### Comparing with competitor's price and factor them into your own pricing

In [26]:
# ---------------MODEL 3---------------

# choosing our lot and competitor lot from existing data, "BHMBCCTHL01" was choosen as our lot
my_lot_id = "BHMBCCTHL01"           # BHMBCCTHL01 was selected as it contains instances where occupancy > capacity to implement our competitive logic
competitor_lot_id = "BHMBCCMKT01"   #assuming this is our competitor

# Filter my lot and competitor lot data from model 2
data_my_lot = model2.filter(pw.this.SystemCodeNumber == my_lot_id)
data_competitor = model2.filter(pw.this.SystemCodeNumber == competitor_lot_id)

# Join based on t
joined = data_my_lot.join(
    data_competitor,
    data_my_lot.t == data_competitor.t
)

# Select required columns
joined_selected = joined.select(
    t = data_my_lot.t,
    SystemCodeNumber = data_my_lot.SystemCodeNumber,
    Occupancy = data_my_lot.Occupancy,
    Capacity = data_my_lot.Capacity,
    MyPrice = data_my_lot.Price_Model2,
    CompetitorPrice = data_competitor.Price_Model2,
    IsFull = data_my_lot.Occupancy >= data_my_lot.Capacity
)

# Apply competitive logic
# If our lot is full and nearby lot is cheaper → reduce price
# If nearby lot is expensive → our price can increase while still being attractive
model3 = joined_selected.with_columns(
    Price_Model3 = pw.apply(
        lambda is_full, my_price, competitor_price: (
            my_price - 1.5 if is_full and competitor_price < my_price else               #1.5 can be changed as per our needs
            competitor_price - 1.5 if not is_full and competitor_price > my_price else
            competitor_price - 1.5 if competitor_price < my_price else
            my_price
        ),
        pw.this.IsFull,
        pw.this.MyPrice,
        pw.this.CompetitorPrice
    )
)
# Uncomment below to see model3 price
# pw.debug.compute_and_print(model3)

# Visualization for MODEL 3
#### **Our Price** vs **CompetitorPrice** vs **Model 3 Price**


In [28]:
from bokeh.models import ColumnDataSource, Band

pn.extension()

# -------- Visualization Function --------
def make_plot_model3(source):
    fig = bokeh.plotting.figure(
        height=300,
        width=900,                            #took larger width to see the difference clearly
        title="Model 3 Prices - BHMBCCTHL01",
        x_axis_type="datetime"
    )

    # Plot MyPrice
    fig.line("t", "MyPrice", source=source, line_width=2, color="red", legend_label="Our initial Price")

    # Plot CompetitorPrice
    fig.line("t", "CompetitorPrice", source=source, line_width=2, color="orange", legend_label="Competitor Price")

    # Plot Price_Model3
    fig.line("t", "Price_Model3", source=source, line_width=2, color="green", legend_label="Price_Model3")

    # Add Area Chart (Band)
    band = Band(
        base='t',
        lower='CompetitorPrice',
        upper='Price_Model3',
        source=source,
        level='underlay',
        fill_alpha=0.2,
        fill_color='yellow',
        line_width=0
    )
    fig.add_layout(band)

    fig.legend.location = "top_left"
    fig.legend.click_policy = "hide"
    fig.xaxis.axis_label = "Time"
    fig.yaxis.axis_label = "Price"

    return fig


m3_plot = model3.plot(make_plot_model3, sorting_col="t")


# Visualize Dashboard
dashboard = pn.Column("# MyPrice vs CompetitorPrice vs Price_Model3", m3_plot)
dashboard


Column
    [0] Markdown(str)
    [1] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [29]:
# Start the Pathway pipeline execution in the background
# - This triggers the real-time data stream processing defined above

pw.run()


Output()